# 003.01g Features - Shift Data to Historical Records

In [1]:
import pathlib
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.mlab as mlab

%matplotlib inline

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
#add the 'src' directory to path to import modules
PROJECT_DIR = pathlib.Path.cwd().resolve().parent
sys.path.append(str(PROJECT_DIR))



pd.set_option('display.max_columns', 1500)
pd.set_option('display.width', 1000)

DATA_DIR = PROJECT_DIR / 'data'
SCOPED_DATA_DIR = DATA_DIR / '03-scoped'
PROCESSED_DATA_DIR = DATA_DIR / '04-processed'
SEMI_MERGED_DIR = DATA_DIR / '05-merged-in-leagues'

### Forming Historical Data based on Game Features

In [2]:
def get_league_seasons(LEAGUES, SEASONS):
    
    df_league_seasons = {}
    for league in LEAGUES:
        for season in SEASONS:
            # Form file path
            load_fp = SCOPED_DATA_DIR / league / str(league + '-scoped-1.csv')
            df_orig = pd.read_csv(load_fp, parse_dates = ['date'], index_col=None)
            df_league_seasons[(league, season)] = df_orig[df_orig['season'] == season]
    return df_league_seasons
    

In [43]:
def get_immediate_subdirs(some_posixpath):
    return list(some_posixpath.glob('*/'))
    
def get_immediate_fps(some_posixpath):
    return [fp for fp in some_posixpath.iterdir() if fp.is_file()]

def get_league_seasons(some_posixpath):
    league_seasons = {}
    leagues = get_immediate_subdirs(some_posixpath)
    for league in leagues:
        league_seasons[league] = get_immediate_fps(league)
    return league_seasons
    

# def get_league_seasons(top_level_dir):
#     leagues = top_level_dir[dir for dir in top_level_dir.]
    
#     df_league_seasons = {}
#     for league in LEAGUES:
#         for season in SEASONS:
#             # Form file path
#             load_fp = SCOPED_DATA_DIR / league / str(league + '-scoped-1.csv')
#             df_orig = pd.read_csv(load_fp, parse_dates = ['date'], index_col=None)
#             df_league_seasons[(league, season)] = df_orig[df_orig['season'] == season]
#     return df_league_seasons

In [44]:
ret = get_league_seasons(SCOPED_DATA_DIR)
ret
# LEAGUES = ['english-premier-league']
# SEASONS = ['2009-2010']

# df_league_seasons = get_league_seasons(LEAGUES, SEASONS)
# write_historical_feature_dfs(df_league_seasons)

[PosixPath('/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-scoped/english-premier-league')]


{PosixPath('/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-scoped/english-premier-league'): [PosixPath('/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-scoped/english-premier-league/english-premier-league-scoped-1.csv')]}

In [39]:
ds = get_immediate_subdirs(SCOPED_DATA_DIR)
ds

[PosixPath('/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-scoped/english-premier-league')]

In [40]:
fps = get_immediate_fps(ds[0])
print(fps)

[PosixPath('/media/david/5C14F53A14F517AA/code/ana_py37/projects/soccer-predictions/data/03-scoped/english-premier-league/english-premier-league-scoped-1.csv')]


In [2]:
def get_teams(df, h_team_col, a_team_col):
    """
    Return a list of unique team names by looking in known column names h and a
    """
    teams = (df[h_team_col].append(df[a_team_col])).unique()
    return teams    

def get_games(df_bc, teams, ha):
    team_games = []
    for team in teams:
        team_game = df_bc[df_bc[ha] == team]
        team_games.append(team_game)
    return team_games


def get_lagged_games(df_bc, games, feature_cols, ha):
    lags = range(1, len(games[0]))
    lagged_team_games = []
    for game in games:
        lagged_df = game.assign(**{f'{ha}_{col}_-{n}': game[col].shift(n) \
                                                  for n in lags \
                                                  for col in feature_cols})
        lagged_team_games.append(lagged_df)
    return lagged_team_games


def form_hhaa(df_bc):
    
    H_TEAM = 'h'
    A_TEAM = 'a'
    
    teams = get_teams(df_bc, H_TEAM, A_TEAM)    
    
    home_team_home_games = get_games(df_bc, teams, H_TEAM)
    away_team_away_games = get_games(df_bc, teams, A_TEAM)
    
    all_feature_cols = [col for col in df_bc.columns if col[1:2] == '_']
    
    h_feature_cols = [col for col in df_bc.columns if col[0:2] == H_TEAM + '_']
    lagged_home_team_home_games = get_lagged_games(df_bc, home_team_home_games,
                                                  h_feature_cols, H_TEAM)
    
    a_feature_cols = [col for col in df_bc.columns if col[0:2] == A_TEAM + '_']
    lagged_away_team_away_games = get_lagged_games(df_bc, away_team_away_games,
                                                  a_feature_cols, A_TEAM)        

    # Join DataFrames together
    hh_games = pd.concat([*lagged_home_team_home_games], axis=0, join='inner', sort=True)
    hh_games.sort_index(inplace=True)

    aa_games = pd.concat([*lagged_away_team_away_games], axis=0, join='inner', sort=True)
    aa_games.sort_index(inplace=True)

    hhaa_games = pd.concat([hh_games, aa_games], axis=1, join='inner', sort=True)
    hhaa_games.sort_index(inplace=True)
    #Drop duplicate columns such as h, a etc
    hhaa_games = hhaa_games.loc[:,~hhaa_games.columns.duplicated()]

    # Re-arrange column sequnce to more intuitive
    all_orig_cols = ['season', 'date', 'h', 'a'] + all_feature_cols # Put result in here !!!!
#     print(all_feature_cols)
#     print(all_cols)
    for col in reversed(all_orig_cols):
        ser = hhaa_games[col]
        hhaa_games.drop(labels= col, axis=1,inplace = True)
        hhaa_games.insert(0, col, ser)
    return hhaa_games


def get_hhaa_team_games_dict(hhaa_games, teams):
    # Get Dataframe by team game home and away
    hhaa_team_games_dict = {}
    #away_team_away_games = []
    for team in teams:
        crit1 = hhaa_games['h'] == team
        crit2 = hhaa_games['a'] == team
        team_game = hhaa_games[crit1 | crit2]
        hhaa_team_games_dict[team] = team_game 
    return hhaa_team_games_dict 


def get_ha_teams(hhaa_team_games_dict):
    
    ha_teams = []
    # Get the indices for the team in question
    for team, hhaa_set in hhaa_team_games_dict.items():
        h_indices = hhaa_set[hhaa_set['h'] == team].index
        a_indices = hhaa_set[hhaa_set['a'] == team].index
        # we want to get the home team away record - designated h_a
        # This means we need to get the record based on the previos time the team played away
        # The indexes we want to add columns to are h_indices
        # The indexes of the times they played away are in a_indices
        # We want to form h_a, so we want to shift the a_indices data down to match the h indices
        # We want tyhe closest lower indices
        shifted_indices = [a_indices[a_indices < number].max() for number in h_indices]
        # We now have
        # team_games[0].loc[h_indices] = team_games[0].loc[shifted_indices][data]
        # We will form the data
        # We want to get the away records so we will grab anything that has got an a_a at the start
        data_cols = [col for col in hhaa_set.columns if col[0:2] == 'a_']
        # the column labeled a_shots is actually the previous away game,
        # so we need to increment all the columns by 1
        numbered_cols = [col.rsplit('_',1)[0] + '_' + str(int(col.split('_')[-1])-1) \
                         for col in data_cols if not col.split('_')[-1].isalpha()]
        # # Now deal with the missing -1 value
        new_minus_one = 'a_' + data_cols[0] + '_-1'
        numbered_cols.insert(0, new_minus_one)
        # # Now we are getting h_a data, so we need to relabel the first a as a h
        renamed_cols = ['h' + col[1:] for col in numbered_cols]
        # ha_cols = list(hhaa_set.columns) + renamed_cols
        # So now we can have a go at joining the data
        ha_set = pd.DataFrame(index=h_indices, columns=renamed_cols)
        # Pandas warning: Passing list-likes to .loc or [] with any missing label will raise
        #KeyError in the future - Think it is coming fro line below
        ha_set.loc[h_indices, renamed_cols] = hhaa_set.loc[shifted_indices][data_cols].values
        ha_teams.append(ha_set)
    return ha_teams


def get_ah_teams(hhaa_team_games_dict):
    
    ah_teams = []
    for team, hhaa_set in hhaa_team_games_dict.items():
        h_indices = hhaa_set[hhaa_set['h'] == team].index
        a_indices = hhaa_set[hhaa_set['a'] == team].index
        shifted_indices = [h_indices[h_indices < number].max() for number in a_indices]
        data_cols = [col for col in hhaa_set.columns if col[0:2] == 'h_']
        numbered_cols = [col.rsplit('_',1)[0] + '_' + str(int(col.split('_')[-1])-1) \
                         for col in data_cols if not col.split('_')[-1].isalpha()]
        new_minus_one = 'h_' + data_cols[0] + '_-1'
        numbered_cols.insert(0, new_minus_one)
        renamed_cols = ['a' + col[1:] for col in numbered_cols]
        ah_set = pd.DataFrame(index=a_indices, columns=renamed_cols)
        ah_set.loc[a_indices, renamed_cols] = hhaa_set.loc[shifted_indices][data_cols].values
        ah_teams.append(ah_set)
    return ah_teams


def get_feature_history(df_league_season):
    
    teams = get_teams(df_league_season, 'h', 'a')

    hhaa_games = form_hhaa(df_league_season)
    hhaa_team_games_dict = get_hhaa_team_games_dict(hhaa_games, teams)
    
    ha_teams = get_ha_teams(hhaa_team_games_dict)
    ah_teams = get_ah_teams(hhaa_team_games_dict)

    ha_df = pd.concat(ha_teams, axis=0).sort_index()
    ah_df = pd.concat(ah_teams, axis=0).sort_index()

    df = pd.concat([hhaa_games, ah_df, ha_df], axis=1, sort=True).sort_index()
    
    return df


def get_feature_cols(df):
    feature_cols = list(set([col[2:] for col in df.columns if col[0:2] == 'h_' or col[0:2] == 'a_']))
    return feature_cols


def get_df_league_seasons(LEAGUES, SEASONS):
    
    df_league_seasons = {}
    for league in LEAGUES:
        for season in SEASONS:
            # Form file path
            load_fp = SCOPED_DATA_DIR / league / str(league + '-scoped-1.csv')
            df_orig = pd.read_csv(load_fp, parse_dates = ['date'], index_col=None)
            df_league_seasons[(league, season)] = df_orig[df_orig['season'] == season]
    return df_league_seasons

        
def write_historical_feature_dfs(df_league_seasons):
    
    for (league, season), df_league_season in df_league_seasons.items():
        feature_cols = get_feature_cols(df_league_season)
        for feature_col in feature_cols:
            keeper_cols = ['season', 'date', 'h', 'a', 'h_'+ feature_col, 'a_' + feature_col]
            df_cut = df_league_season[keeper_cols]
            df = get_feature_history(df_cut)

            fn = str(league + '--' + season + '--' + feature_col + '.csv')
            save_fp = PROCESSED_DATA_DIR / league / season / fn
            df.to_csv(save_fp, index=False)


In [3]:
LEAGUES = ['english-premier-league']
SEASONS = ['2009-2010']

df_league_seasons = get_df_league_seasons(LEAGUES, SEASONS)
write_historical_feature_dfs(df_league_seasons)

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:115: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/david/anaconda3/envs/ana_py37/lib/python3.7/site-packages/ipykernel_launcher.py:134: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://panda

In [4]:
# Load up and check they look OK
# Compile in the next Notebook
def get_historical_feature_dfs(LEAGUES, SEASONS):
    league_season_dfs = []
    for league in LEAGUES:
        for season in SEASONS:
            league_season_dir = PROCESSED_DATA_DIR / league / season
            fps = [fp for fp in league_season_dir.glob('**/*') if fp.is_file()]
            for fp in fps:
                #print(fp)
                league_season_dfs.append(pd.read_csv(fp, parse_dates = ['date'], index_col=None))
    hist_feature_df = pd.concat(league_season_dfs, axis=1, sort=True)
    #Drop duplicate columns such as h, a etc
    hist_feature_df = hist_feature_df.loc[:,~hist_feature_df.columns.duplicated()]
    return hist_feature_df
            
#     fn = str(league + '--' + season + '--' + feature_col + '.csv')
#     df_league_seasons = {}
#     for league in LEAGUES:
#         for season in SEASONS:
#             # Form file path
#             load_fp = SCOPED_DATA_DIR / league / str(league + '-scoped-1.csv')
#             df_orig = pd.read_csv(load_fp, parse_dates = ['date'], index_col=None)
#             df_league_seasons[(league, season)] = df_orig[df_orig['season'] == season]
# #             df_league_seasons.append(df_orig[df_orig['season'] == season])
#     return df_league_seasons

        
# def write_historical_feature_dfs(df_league_seasons):
#     for (league, season), df_league_season in df_league_seasons.items():
#         feature_cols = get_feature_cols(df_league_season)
#         for feature_col in feature_cols:
#             keeper_cols = ['season', 'date', 'h', 'a', 'h_'+ feature_col, 'a_' + feature_col]
#             df_cut = df_league_season[keeper_cols]
#             df = get_feature_history(df_cut)

#             fn = str(league + '--' + season + '--' + feature_col + '.csv')
#             save_fp = PROCESSED_DATA_DIR / league / season / fn
#             df.to_csv(save_fp, index=False)

            
LEAGUES = ['english-premier-league']
SEASONS = ['2009-2010']

league_season_df = get_historical_feature_dfs(LEAGUES, SEASONS)
league_season_df.tail(10)
# write_historical_feature_dfs(df_league_seasons)

season       date                        h                        a  h_corners  a_corners  h_h_corners_-1  h_h_corners_-10  h_h_corners_-11  h_h_corners_-12  h_h_corners_-13  h_h_corners_-14  h_h_corners_-15  h_h_corners_-16  h_h_corners_-17  h_h_corners_-18  h_h_corners_-2  h_h_corners_-3  h_h_corners_-4  h_h_corners_-5  h_h_corners_-6  h_h_corners_-7  h_h_corners_-8  h_h_corners_-9  a_a_corners_-1  a_a_corners_-10  a_a_corners_-11  a_a_corners_-12  a_a_corners_-13  a_a_corners_-14  a_a_corners_-15  a_a_corners_-16  a_a_corners_-17  a_a_corners_-18  a_a_corners_-2  a_a_corners_-3  a_a_corners_-4  a_a_corners_-5  a_a_corners_-6  a_a_corners_-7  a_a_corners_-8  a_a_corners_-9  a_h_corners_-1  a_h_corners_-2  a_h_corners_-11  a_h_corners_-12  a_h_corners_-13  a_h_corners_-14  a_h_corners_-15  a_h_corners_-16  a_h_corners_-17  a_h_corners_-18  a_h_corners_-19  a_h_corners_-3  a_h_corners_-4  a_h_corners_-5  a_h_corners_-6  a_h_corners_-7  a_h_corners_-8  a_h_corners_-9  \
355  2009-2010 2010-04-25                  burnley                liverpool        4.0        5.0             6.0              7.0              6.0              8.0              7.0              8.0              4.0              3.0              1.0              NaN             7.0            11.0             4.0             4.0             3.0             2.0             8.0             3.0             4.0              8.0              4.0              2.0              7.0              8.0              4.0             11.0              5.0              NaN             2.0             7.0             5.0             6.0             7.0             0.0            10.0            10.0             9.0            10.0              3.0              7.0             11.0              5.0             13.0              7.0             10.0             11.0              NaN             9.0             8.0             4.0             8.0            11.0             2.0             7.0   
356  2009-2010 2010-04-25                  chelsea               stoke-city        7.0        0.0             5.0              7.0              5.0              0.0              8.0              5.0             11.0             10.0             12.0              NaN             8.0            11.0             3.0             4.0             8.0            15.0            10.0            11.0             6.0              2.0              2.0              1.0              4.0              6.0              3.0              5.0              NaN              NaN             7.0             4.0             7.0             5.0             3.0             6.0             4.0             2.0             5.0             3.0              5.0              4.0              4.0              5.0              2.0              4.0             10.0              3.0              NaN             6.0             8.0             3.0             3.0            12.0            10.0             8.0   
357  2009-2010 2010-04-25                  everton                   fulham        9.0        3.0             2.0              1.0              6.0              3.0              6.0             11.0              6.0             12.0              4.0              NaN             6.0             4.0             4.0             5.0             3.0            11.0            10.0             6.0             1.0              4.0              7.0              6.0              2.0              4.0              3.0              4.0              4.0              NaN             8.0             1.0             2.0             5.0             3.0             6.0            10.0             6.0             5.0             6.0              8.0              3.0              2.0              7.0              4.0              5.0              4.0              NaN              NaN             4.0             3.0             6.0             5.0             9.0             4.0             5.0   
358  2009-2010 2010-05-01          manch

In [5]:
league_season_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 380 entries, 0 to 379
Data columns (total 1068 columns):
season                           380 non-null object
date                             380 non-null datetime64[ns]
h                                380 non-null object
a                                380 non-null object
h_corners                        380 non-null float64
a_corners                        380 non-null float64
h_h_corners_-1                   360 non-null float64
h_h_corners_-10                  180 non-null float64
h_h_corners_-11                  160 non-null float64
h_h_corners_-12                  140 non-null float64
h_h_corners_-13                  120 non-null float64
h_h_corners_-14                  100 non-null float64
h_h_corners_-15                  80 non-null float64
h_h_corners_-16                  60 non-null float64
h_h_corners_-17                  40 non-null float64
h_h_corners_-18                  20 non-null float64
h_h_corners_-2               

In [6]:
save_fp = SEMI_MERGED_DIR / LEAGUES[0] / str(LEAGUES[0] + '.csv')
league_season_df.to_csv(save_fp, index=False)

In [7]:
stop

NameError: name 'stop' is not defined

In [ ]:
df.head(20)

In [ ]:
df.tail(20)

In [ ]:
df.info(verbose=True)

In [ ]:
stop

In [ ]:
df_load = pd.read_csv(save_fp, parse_dates = ['date'], index_col=None)
df_load.head()

In [ ]:
df_load.info()

## Functions to Cut Data to Defined Shape prior to PreProcessing

In [ ]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(past_games+1) \
               or col.isalpha() \
               or 'odds' in col]
    keeper_cols = keeper_cols + ['result']#, 'h_ftgoals', 'a_ftgoals']
    
#     # Rename the target columns to start with prefix 'res_'
#     res_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
#                 and int(col.split('_')[-1]) == 0]
#     res_cols.append('result')
#     res_cols = [col.replace('0', 'res') for col in res_cols]
#     print(res_cols)
#     print(res_cols)
    df = df[keeper_cols]
    return df

df_load = pd.read_csv(save_fp, parse_dates = ['date'], index_col=None) 
df_cut = get_n_past_games(df_load)
# # Restrict Historical records to last 4 games per team
# df_cut = df_load.copy(deep=True)
# hist_games = 2
# keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
#                and int(col.split('_')[-1]) > -(hist_games+1) \
#                and int(col.split('_')[-1]) < 0 \
#                or col.isalpha() \
#                or 'odds' in col]
# df_cut = df_cut[keeper_cols]
df_cut.head()

In [ ]:
df_cut.tail()

In [ ]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
    

# Restrict Historical records to last 4 games per team
df_cut = df_load.copy(deep=True)
hist_games = 2
keeper_cols = [col for col in df.columns if not col.split('_')[-1].isalpha() \
               and int(col.split('_')[-1]) > -(hist_games+1) \
               and int(col.split('_')[-1]) < 0 \
               or col.isalpha() \
               or 'odds' in col]
df_cut = df_cut[keeper_cols]
df_cut.head(10)